In [ ]:
import sys
sys.path.append('../')

In [ ]:
from game.agent import RandomAgent
from game.simulator import YarnSimulator
from pathlib import Path
from tqdm.auto import trange
import pandas as pd
# import itertools
import mpire

In [ ]:
n_runs = int(1e6)
seed = None

In [ ]:
graph = YarnSimulator(text_unk_macro="", yarn='../yarnScripts').get_decision_graph()[0]

In [ ]:
endings = []

def init_run(worker_state):
    # Create simulator
    worker_state['simulator'] = YarnSimulator(text_unk_macro="", yarn='../yarnScripts')
    # worker_state['endings'] = []
    pass

def run_game(worker_state, run):
    simulator = worker_state['simulator']
    simulator.restart()
    # simulator = YarnSimulator(text_unk_macro="", yarn='../yarnScripts')
    agent = RandomAgent(seed=seed)
    
    final = None
    # run simulator
    state, choices, extras = simulator.read()
    while not simulator.is_finished():
        # print(extras)
        # print(simulator.controller.state.attr)

        if (tmp := simulator.controller.state.attr.get('final', None)) is not None:
            final = (simulator.controller.state.title, tmp)

        choice = agent.act(state, choices)
        state, choices, extras = simulator.transition(choice)
    
    # worker_state['endings'] = worker_state['endings'].append(final)
    # worker_state['endings'].append(final)
    endings.append(final)
    return final

def exit_func(worker_state):
    # Return the list
    # return worker_state['endings']
    pass

with mpire.WorkerPool(n_jobs=None, use_worker_state=True, start_method="threading") as pool:
    pool.map(run_game, range(n_runs), worker_init=init_run)#, worker_exit=exit_func)
    # print(pool.get_exit_results())
    # endings = list(itertools.chain.from_iterable(pool.get_exit_results()))
    df = pd.DataFrame(data=endings, columns=['title', 'kind'])
    print(df)
    # df.write_excel('results.xlsx')


## Random Agent

In [ ]:
simulator = YarnSimulator(text_unk_macro="", yarn='../yarnScripts')

endings = []
for k in trange(int(n_runs)):
    # reset simulator and agent
    simulator.restart()
    agent = RandomAgent(seed=seed)
    # agent = TorchModelAgent(
    #     model_path=Path('../tmp/adamw_max_val_acc_8_False_125,[20],[20]_0.001'),
    #     rand=0,
    #     use_cpu=False,
    #     seed=seed,
    # )

    final = None
    # run simulator
    state, choices, extras = simulator.read()
    while not simulator.is_finished():
        # print(extras)
        # print(simulator.controller.state.attr)

        if (tmp := simulator.controller.state.attr.get('final', None)) is not None:
            final = (simulator.controller.state.title, tmp)

        choice = agent.act(state, choices)
        state, choices, extras = simulator.transition(choice)
    
    endings.append(final)

df = pd.DataFrame(data=endings, columns=['title', 'kind'])
        
print(df)


In [ ]:
df.to_csv('endings.csv')

In [ ]:
df['kind'].value_counts(normalize=True)

In [ ]:
def init_func(worker_state):
    # Initialize a counter for each worker
    # worker_state['count_even'] = 0
    pass

def square_and_count_even(worker_state, x):
    # Count number of even numbers and return the square
    # if x % 2 == 0:
    #     worker_state['count_even'] += 1
    # return x * x
    pass

def exit_func(worker_state):
    # Return the counter
    # return worker_state['count_even']
    pass

with mpire.WorkerPool(n_jobs=4, use_worker_state=True, start_method="spawn") as pool:
    pool.map(square_and_count_even, range(100), worker_init=init_func, worker_exit=exit_func, progress_bar=True)
    print(pool.get_exit_results())  # Output, e.g.: [13, 13, 12, 12]
    print(sum(pool.get_exit_results()))  # Output: 50


## Model Agent

In [ ]:
graph = YarnSimulator(text_unk_macro="", yarn='../yarnScripts').get_decision_graph()[0]

In [ ]:
graph.out_edges('Start$sub78_{}', data=True)

In [ ]:
graph['Start$sub78_{}']

In [ ]:
# {v:k for k,v in graph['Start$sub78_{}'].items()}

{v['action']:{'node':k, 'extras':v['extras']} for k,v in graph['Start$sub78_{}'].items()}

In [ ]:
graph.out_degree['Start']

In [ ]:
graph.nodes(data=True)['Start']